# Create Captions with OpenCLIP

This is based on the Colab, [here](https://colab.research.google.com/github/mlfoundations/open_clip/blob/master/docs/Interacting_with_open_coca.ipynb)

In [1]:
!pip install open_clip_torch transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.7 MB/s eta 0:00:00


# Get input images and texts

We are going to feed 8 example images and their textual descriptions to the model, and compare the similarity between the corresponding features.

In [47]:
data = [
["https://live.staticflickr.com/65535/48545607901_8200f85f06.jpg", "photos/48545607901_8200f85f06.jpg", ""],
["https://live.staticflickr.com/2106/2178387761_e6e63fb095.jpg", "photos/2178387761_e6e63fb095.jpg", ""],
["https://live.staticflickr.com/7795/26246790753_bf28b47e74.jpg", "photos/26246790753_bf28b47e74.jpg", ""],
["https://live.staticflickr.com/65535/52639014073_1d7e09afac.jpg", "photos/52639014073_1d7e09afac.jpg", ""],
["https://live.staticflickr.com/4554/38290258711_c17067b136.jpg", "photos/38290258711_c17067b136.jpg", ""],
["https://live.staticflickr.com/813/26022621907_075ae5ecd2.jpg", "photos/26022621907_075ae5ecd2.jpg", ""]
]


In [36]:
# from google.colab import files
# # Set the path to the file you want to upload
# local_path = "/Users/komal/Downloads/logo.jpg"

# # Set the name of the folder you want to upload the file to
# folder_name = "photos"

# # Use the files.upload() function to upload the file to Colab
# uploaded = files.upload()


Saving logo.jpg to logo (1).jpg


In [48]:
import urllib.request
import textwrap
import os

path_names = []
descriptions = []

# Create a folder to save the photos
folder = "photos"
if not os.path.exists(folder):
  os.makedirs(folder)

for d in data:
  # Retrieve the list of photos
  # url = d[0]
  caption = d[2]
  caption = caption.replace("(LOC)", "").replace("[", "").replace("]","").strip()
  path_name = d[1]
  # urllib.request.urlretrieve(url, path_name)
  path_names.append(path_name)
  descriptions.append(caption)
  print(path_name, caption)
print(path_names)

photos/48545607901_8200f85f06.jpg 
photos/2178387761_e6e63fb095.jpg 
photos/26246790753_bf28b47e74.jpg 
photos/52639014073_1d7e09afac.jpg 
photos/38290258711_c17067b136.jpg 
photos/26022621907_075ae5ecd2.jpg 
['photos/48545607901_8200f85f06.jpg', 'photos/2178387761_e6e63fb095.jpg', 'photos/26246790753_bf28b47e74.jpg', 'photos/52639014073_1d7e09afac.jpg', 'photos/38290258711_c17067b136.jpg', 'photos/26022621907_075ae5ecd2.jpg']


In [ ]:
import open_clip
import torch

model, _, transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)
model.cuda().eval()

In [49]:
from PIL import Image
new_captions = []
for path_name in path_names:
  im = Image.open(path_name).convert("RGB")
  im = transform(im).unsqueeze(0).cuda()
  with torch.no_grad(), torch.cuda.amp.autocast():
    generated = model.generate(im)
  caption = open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", "")[:-2]
  print(caption)
  new_captions.append(caption)

a mexican food sign on the side of a building 
a group of people standing next to a brick building 
an old photo of a man in a military unifor
a man kneeling down next to a dog 
an old tea pot shaped building on the side of the street 
a painting of a young woman with a black shirt on 
